# PREPROCESSING

The purpose of this Notebook is to split all 1km x 1km regions stored as .tif-files into smaller tiles that are suitable as network inputs. Tiles will be saved as .jpg-images in a new folder along with point-labels and shape-labels as .npy-files.

The final structure of the output folder will look like this:

Folder name: 256x265 (width x hight of tiles) 
- images: contains individual tiles as .jpg-images with name {region name} _ {tile number}.jpg 
- points: point-labels as .npy-files, matched via filename
- shapes: shape-labels as .npy-files, matched via filename
- images_sets: contains a .txt-file for each split (train, val, test) with the filenames of the assigned tiles
   

In [1]:
import os
import math
import fiona
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
import rasterio as rio
from shapely.geometry import Point, Polygon

## Get all Images and Label-Files from Directory

In [2]:
DATADIR = "/home/jovyan/work"
DATASET = "DENMARK"
IMAGETYPE = ".tif"
LABELTYPE = ".shp"

regions = []
labels = []
PATH = os.path.join(DATADIR, DATASET)
for _, _, files in os.walk(PATH):
    for file in files:
        if file.endswith(IMAGETYPE):
            regions.append(file)
        elif file.endswith(LABELTYPE):
            labels.append(file)
            
print(f"Found {len(regions)} Regions and {len(labels)} Label-Files")

Found 2 Regions and 2 Label-Files


## Set Tile Size and Overlap

To ensure equal tile sizes, overap is computed dynamically based on the amount of vertical and horizontal tiles

In [3]:
tiles_h = 40
tiles_v = 40
width = 256
height = 256

example_src = rio.open(os.path.join(PATH, regions[0]))
ncols, nrows = example_src.meta['width'], example_src.meta['height']
h_overlap = ((tiles_h * width) - ncols) / (tiles_h - 1)
v_overlap = ((tiles_v * height) - nrows) / (tiles_v - 1)

print(f"Generating {tiles_h * tiles_v} tiles per region with: \n - tile size: {width} x {height} px \n - region size: {ncols} x {nrows} px \n - vertical overlap: {v_overlap} px \n - horizontal overlap: {h_overlap} px")

Generating 1600 tiles per region with: 
 - tile size: 256 x 256 px 
 - region size: 8000 x 8000 px 
 - vertical overlap: 57.43589743589744 px 
 - horizontal overlap: 57.43589743589744 px


## Generate Collection

Each entry contains the OG file, number of tile and tile image as RGB-array

In [4]:
point_labels = []
shape_labels = []
for file in labels:
    with fiona.open(os.path.join(PATH, file)) as shapefile:
        for feature in shapefile:
            if feature['geometry']['type'] == "Point":
                point = feature["geometry"]['coordinates'][:2]
                x = point[0]
                y = point[1]
                point_labels.append([Point(point), x, y])
            elif feature['geometry']['type'] == "Polygon":
                shape = feature["geometry"]['coordinates'][0]
                x_min, x_max = min(shape, key = lambda t: t[0])[0], max(shape, key = lambda t: t[0])[0]
                y_min, y_max = min(shape, key = lambda t: t[1])[1], max(shape, key = lambda t: t[1])[1]
                shape_labels.append([Polygon(shape), x_min, y_min, x_max, y_max])
        
point_labels = pd.DataFrame(data=point_labels, columns=["Point", "X", "Y"])
shape_labels = pd.DataFrame(data=shape_labels, columns=["Shape", "Xmin", "Ymin", "Xmax", "Ymax"])
print(f"Found {len(point_labels)} Point Labels and {len(shape_labels)} Shape Labels")

Found 1133 Point Labels and 602 Shape Labels


Iterate over Polygon-Shapes and assign to points that they cover

In [5]:
for index, row in shape_labels.iterrows():
    shape = row["Shape"]
    point_labels.loc[point_labels['Point'].apply(lambda p: p.within(shape)), "Shape"] = shape

In [7]:
collection = []

for region in tqdm(regions):
    src = rio.open(os.path.join(PATH, region))
    name_clean = region.replace(".tif","")
    
    # region as window and shapely polygon
    ncols, nrows = src.meta['width'], src.meta['height']
    bounds = list(src.bounds)
    big_window = rio.windows.Window(col_off = 0, row_off = 0, width = ncols, height = nrows)
    big_poly = Polygon([(bounds[0], bounds[1]), (bounds[2], bounds[1]), (bounds[2], bounds[3]), (bounds[0], bounds[3])])
    
    # filter X and Y
    region_points = point_labels[point_labels['Point'].apply(lambda p: p.within(big_poly))].copy()
    # region_points = point_labels[(point_labels.X > bounds[0]) & (point_labels.X < bounds[2])]
    # region_points = region_points[(region_points.Y > bounds[1]) & (region_points.Y < bounds[3])]
    
    # cut shapes to region bounds
    region_points['Shape'] = region_points['Shape'].apply(lambda s: s.intersection(big_poly) if not isinstance(s, float) else s)
    region_points['ShapeX'] = region_points['Shape'].apply(lambda s: s.exterior.coords.xy[0] if not isinstance(s, float) else s)
    region_points['ShapeY'] = region_points['Shape'].apply(lambda s: s.exterior.coords.xy[1] if not isinstance(s, float) else s)
    
    # translate to pixels
    region_points['X'] = region_points['X'].apply(lambda x: (x - bounds[0])*8) #CHANGE
    region_points['Y'] = region_points['Y'].apply(lambda y: (1000 - (y - bounds[1]))*8)
    region_points['ShapeX'] = region_points['ShapeX'].apply(lambda lx: [(x - bounds[0])*8 for x in lx] if not isinstance(lx, float) else lx)
    region_points['ShapeY'] = region_points['ShapeY'].apply(lambda ly: [(1000 - (y - bounds[1]))*8 for y in ly] if not isinstance(ly, float) else ly)
    
    # traverse tiles column bv column, row by row
    for row in tqdm(range(tiles_v)):
        row_off = int(row * (height - v_overlap))
        for col in range(tiles_v):
            col_off = int(col * (width - h_overlap))
            # read tile part of region
            window = rio.windows.Window(col_off = col_off, row_off = row_off, width = width, height = height).intersection(big_window)
            src_image = src.read(window = window)[:3]
            image = np.stack((src_image[0], src_image[1], src_image[2]), axis = 2)
            # get points in tile 
            tile_points = region_points[(region_points.X > col_off) & (region_points.X < (col_off+width))]
            tile_points = tile_points[(tile_points.Y > row_off) & (tile_points.Y < (row_off+height))]
            # translate to new dimensions
            tile_points['X'] = tile_points['X'].apply(lambda x: round(x - col_off))
            tile_points['Y'] = tile_points['Y'].apply(lambda y: round(y - row_off))
            tile_points['ShapeX'] = tile_points['ShapeX'].apply(lambda lx: [round(x - col_off) for x in lx] if not isinstance(lx, float) else lx)
            tile_points['ShapeY'] = tile_points['ShapeY'].apply(lambda ly: [round(y - row_off) for y in ly] if not isinstance(ly, float) else ly)
            # put shape coordinates together
            tile_points['pShape'] = list(zip(tile_points.ShapeX, tile_points.ShapeY))
            tile_points['pShape'] = tile_points['pShape'].apply(lambda t: list(zip(t[0], t[1])) if not isinstance(t[0], float) else float('NaN'))
            # to Numpy
            np_points = tile_points[['X', 'Y']].to_numpy()
            np_shapes = tile_points['pShape'].dropna().to_numpy()
            # add to collection
            collection.append({"file": name_clean, "tile": str(col + (row * tiles_v) + 1), "image": image, "points": np_points, "npoints": len(np_points), "shapes": np_shapes, "nshapes": len(np_shapes)})

collection = pd.DataFrame(collection)
print(f"Generated Collection of {len(collection)} Tiles")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

Generated Collection of 3200 Tiles


## Write To Data Directory

Images...

In [ ]:
TARGET_TYPE = ".jpg"
#make new folder in data directory
NEW_PATH = os.path.join(PATH, f"{width}x{height}")
os.mkdir(NEW_PATH)
IMAGE_PATH = os.path.join(NEW_PATH, "images")
os.mkdir(IMAGE_PATH)

for index, item in collection.iterrows():
    data = item['image']
    name = item['file'] + "_" + item['tile'] + TARGET_TYPE
    #np.save(os.path.join(IMAGE_PATH, name), data) #change TARGET_TYPE
    img = Image.fromarray(data, 'RGB')
    img.save(os.path.join(IMAGE_PATH, name))

Point-Labels...

In [9]:
LABEL_PATH = os.path.join(NEW_PATH, "points")
os.mkdir(LABEL_PATH)
items_with_label = collection[collection.npoints > 0]
for index, item in items_with_label.iterrows():
    data = item['points']
    name = item['file'] + "_" + item['tile'] + "_points.npy"
    np.save(os.path.join(LABEL_PATH, name), data)

Polygon-Shapes...

In [10]:
SHAPE_PATH = os.path.join(NEW_PATH, "shapes")
os.mkdir(SHAPE_PATH)
items_with_shape = collection[collection.nshapes > 0]
for index, item in items_with_shape.iterrows():
    data = item['shapes']
    name = item['file'] + "_" + item['tile'] + "_shapes.npy"
    np.save(os.path.join(SHAPE_PATH, name), data)

## Define Train, Val and Test Sets

In [11]:
SETS_PATH = os.path.join(NEW_PATH, "image_sets")
os.mkdir(SETS_PATH)

Filter collection to e.g. only include images in training that feature at least one point

In [13]:
all_file = open(os.path.join(SETS_PATH, "all.txt"), "w")
points_file = open(os.path.join(SETS_PATH, "points.txt"), "w") 
shapes_file = open(os.path.join(SETS_PATH, "shapes.txt"), "w") 

for index, item in collection.iterrows():
    name = item['file'] + "_" + item['tile'] + "\n"
    all_file.write(name)
    if item.nshapes > 0:
        shapes_file.write(name)
    if item.npoints > 0:
        points_file.write(name) 

all_file.close() 
points_file.close()     
shapes_file.close()

# CONVOLUTIONAL ORIENTED BOUNDARIES

Generates COB-Images for Dataset

In [14]:
import os
import torch
import numpy as np
from tqdm.notebook import tqdm
from skimage.io import imread
from imgaug import augmenters as iaa
from models.base.cobnet import CobNet
from imgaug.augmenters import Augmenter
from torchvision.utils import save_image

from helpers.cob.dataset import rescale_augmenter, Normalize

## Enter Settings and Search Files

In [15]:
PATH = "/home/jovyan/work/DENMARK/256x256"
IMAGES_PATH = os.path.join(PATH, "images")
SAVE_PATH = os.path.join(PATH, "cob")

TYPE = ".jpg"
IMAGE_WIDTH = 256
STATE_DICT = "/home/jovyan/work/runs/COBNET/cp_or.pth.tar"

In [16]:
images = []

for _, _, files in os.walk(IMAGES_PATH):
    for file in files:
        if file.endswith(TYPE):
            images.append(file)

n_images = len(images)
print(f"Found {n_images} images")

Found 3200 images


## Normalize

In [20]:
images_normalized = []

normalization_mean = [0.492, 0.475, 0.430]
normalization_std= [0.176, 0.173, 0.176]

aug = iaa.Sequential([
    iaa.size.Resize(IMAGE_WIDTH),
    rescale_augmenter,
    Normalize(mean = normalization_mean, std = normalization_std)
])

aug_det = aug.to_deterministic()

for i in tqdm(range(n_images)):
    image = imread(os.path.join(IMAGES_PATH, images[i]))
    image = aug_det(images = image[np.newaxis, ...])[0]
    image = np.stack((image[:,:,0], image[:,:,1], image[:,:,2]), axis = 0)
    #image = image[np.newaxis, ...] either create batch axis here, at tensor creation or use batches
    images_normalized.append(image)

#batch = np.stack(img_collection, axis = 0)

  0%|          | 0/3200 [00:00<?, ?it/s]

## COB Model Runs

In [21]:
cob_collection = []

model = CobNet()
model.load_state_dict(torch.load(STATE_DICT))

for image in tqdm(images_normalized):
    img_tensor = torch.tensor(image[np.newaxis, ...]).float()
    model.eval()
    with torch.no_grad():
        cob = model(img_tensor)
    cob_collection.append(cob)
    
print(f"Generated {len(cob_collection)} COB images.")

  0%|          | 0/3200 [00:00<?, ?it/s]

Generated 3200 COB images.


## Save

In [22]:
os.mkdir(SAVE_PATH) # comment out if existing
cob_file = open(os.path.join(PATH, "image_sets", "cob.txt"), "w")

for i in range(len(cob_collection)):
    data = cob_collection[i]['y_fine'].sigmoid()
    path = os.path.join(SAVE_PATH, images[i])
    cob_file.write(name) 
    save_image(data, path)
    
cob_file.close()

# EXPLORATION ZONE

In [ ]:
# How many 250x250 tiles actually have a label?
have = len(collection[collection.npoints > 0])
total = len(collection)
print(f"From a total of {total} tiles, {have} have a label assigned ({have/total*100} %)")